In [ ]:
!pip install konlpy
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 13.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661618 sha256=cfe1dde39caeb09ada3087816cd353122ad1d2926f1fff593dc32d4271301956
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.5 MB/s eta 0:00:00
     ━━━

In [ ]:
from konlpy.tag import Komoran
komoran = Komoran()

from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model = AutoModelForMaskedLM.from_pretrained("monologg/kobert")

Some weights of BertForMaskedLM were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import re
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from konlpy.tag import Komoran
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
import itertools
import numpy as np
import time, datetime
import matplotlib.pyplot as plt

In [ ]:
# 문단을 문장으로 나누는 함수
def sentence_tokenizer(paragraph):
    return re.split(r'(?<=[.!?])\s+', paragraph)

# 문장 어절 단위로 토큰화
def word_tokenizer(sentence):
    return sentence.split(' ')

# 따옴표와 따옴표를 포함하는 문장을 하나의 토큰으로
def process_quoted_words(tokens):
    processed_tokens = []
    quoted_word = ""
    in_quote = False

    for token in tokens:
        if "'" in token:
            if in_quote:
                quoted_word += " " + token
                processed_tokens.append(quoted_word)
                quoted_word = ""
                in_quote = False
            else:
                quoted_word = token
                in_quote = True
        else:
            if in_quote:
                quoted_word += " " + token
            else:
                processed_tokens.append(token)
    return processed_tokens

# 형태소 분석 후 문법적 중요도 점수 추가
def add_linguistic_score(sentence):
    pos = komoran.pos(sentence)
    score = 0
    for p in pos:
        if p[1] == 'NNP':
            score += 0.000001
        elif p[1] == 'NNG':
            score += 0.000001
        elif p[1] == 'vv':
            score += 0.000001
        elif p[1] == 'SL':
            score += 0.000001
        elif p[1] == 'JKS':
            score += 00.000001
        elif p[1] == 'JKO':
            score += 0.000001
    quoted_words = re.findall(r"'(.*?)'", sentence)
    for quoted_word in quoted_words:
        score += 0.0001
    return score

# KoBERT 모델을 이용하여 perplexity 계산
def calculate_perplexity_score(sentence):
    # KoBERT 모델이 이해할 수 있는 형태로 tokenize
    tokens = tokenizer.tokenize(sentence)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    # [MASK] 토큰 위치 찾기
    mask_token_index = token_ids.index(tokenizer.mask_token_id)

    # 입력값 생성
    input_ids = torch.tensor([token_ids])
    outputs = model(input_ids)
    predictions = outputs[0]

    # [MASK] 토큰 위치에 대한 예측값 추출
    masked_predictions = predictions[0, mask_token_index]

    # softmax 함수를 이용하여 확률값을 확률 분포로 변환
    probs = torch.softmax(masked_predictions, dim=-1)

    # perplexity 계산
    perplexity = torch.exp(torch.mean(torch.log(probs)))

    return perplexity.item()

# 압축 문장 후보 생성 및 문법적 중요도 점수 계산
def compress_sentence(token):
    compressed_candidates = []
    max_n = 4 #len(token) - 4

    for n in range(1, max_n + 1):
        for i in range(len(token) - n + 1):
            compressed_tokens = token[:i] + token[i + n:]
            compressed_sentence = " ".join(compressed_tokens)
            score = add_linguistic_score(compressed_sentence)
            compressed_candidates.append((compressed_sentence, score, n))

    perplexity_scores = []
    compressed_candidates_with_score = []

    for n in range(1, max_n + 1):
        for i in range(len(token) - n + 1):
            mask_idx = list(range(i, i + n))
            masked_tokens = list(token)
            for j in mask_idx:
                masked_tokens[j] = "[MASK]"
            masked_sentence = " ".join(masked_tokens)

            perplexity_score = calculate_perplexity_score(masked_sentence)
            linguistic_score = compressed_candidates[i][1]
            final_score = perplexity_score - linguistic_score

            perplexity_scores.append(perplexity_score)
            compressed_candidates_with_score.append((re.sub(r'\[MASK\]\s*', '', masked_sentence), final_score, n))

    compressed_candidates_with_score_sorted = sorted(compressed_candidates_with_score, key=lambda x: x[1])
    final_compressed_sentence = re.sub(r'\[MASK\]\s*', '', compressed_candidates_with_score_sorted[0][0])
    selected_n = compressed_candidates_with_score_sorted[0][2]

    return compressed_candidates_with_score_sorted, final_compressed_sentence, selected_n

# 문단을 한 문장씩 나눠서 압축하고, 최종 결과를 한 문단으로 합치는 함수
def compress_paragraph(paragraph):
    sentences = sentence_tokenizer(paragraph)
    compressed_sentences = []

    for sentence in sentences:
        token = word_tokenizer(sentence)
        token = process_quoted_words(token)
        compressed_candidates_with_score_sorted, final_compressed_sentence, selected_n = compress_sentence(token)
        compressed_sentences.append(final_compressed_sentence)

    compressed_paragraph = ' '.join(compressed_sentences)
    return compressed_paragraph

# 입력 문단
paragraph = input()

# 문단을 압축한 결과
compressed_paragraph = compress_paragraph(paragraph)

# 결과 출력
print("입력 문단:")
print(paragraph)

print("\n최종 요 문단:")
print(compressed_paragraph)

고급 식자재로 불리는 킹크랩 가격이 4년 만에 1㎏당 7만원대로 떨어졌다. 러시아‧우크라이나 전쟁으로 인한 물량 증가 때문이다. 12일 수산물 유통 플랫폼 인어교주해적단에 따르면 이달 10일 러시아산 레드 킹크랩의 평균 가격은 ㎏당 7만400원으로 3개월 전인 7월 10일과 비교해 41.3% 떨어졌다. 지난달 18일까지 kg 당 11만5000원이던 레드 킹크랩 가격은 하루 만에 7만7400원으로 4만원 가까이 하락했다. 이에 따라 한때 30만원까지 치솟았던 킹크랩 한 마리 가격도 17만5000원으로, 20만원 밑으로 하락했다. 킹크랩 시세 하락 배경으로는 러시아‧우크라이나 전쟁으로 인한 물량 증가가 꼽힌다. 전쟁 이후 미국과 유럽이 러시아산 해산물 수입을 금지하면서 러시아는 자국 냉동창고에 킹크랩을 보관했는데, 그 냉동창고가 포화 상태에 이르렀다고 한다. 올해 9월 첫 조업을 앞두고 아시아 국가로의 수출 비중을 크게 늘렸다고 전문가들은 분석한다. 다만, kg 당 가격만을 생각했다간 실제 구입 시에 킹크랩 가격 하락을 크게 체감하지 못할 수 있다. 킹크랩 한 마리는 보통 2.5kg 이상으로, 여기에 찜비나 손질비 등 추가 비용을 합하면 20만원 정도를 내야 한다. 또한 수산물은 산지도매시장이나 소비지도매시장에서 매일 경매를 통해 가격이 결정된다. 가격 변동 폭이 크기 때문에 현재 하락세가 언제까지 지속될지는 장담할 수 없다. 킹크랩 가격 하락 소문을 듣고 사람들이 몰리면 수요가 증가해 킹크랩 가격이 오를 수 있다는 뜻이다.
입력 문단:
고급 식자재로 불리는 킹크랩 가격이 4년 만에 1㎏당 7만원대로 떨어졌다. 러시아‧우크라이나 전쟁으로 인한 물량 증가 때문이다. 12일 수산물 유통 플랫폼 인어교주해적단에 따르면 이달 10일 러시아산 레드 킹크랩의 평균 가격은 ㎏당 7만400원으로 3개월 전인 7월 10일과 비교해 41.3% 떨어졌다. 지난달 18일까지 kg 당 11만5000원이던 레드 킹크랩 가격은 하루 만에 7만7400원으로 4만원 가까이 하락했다. 이에 따라